---
title: 'Total magnetic fields: Dipole and Pole sources'
authors:
  - name: Seogi Kang
    github: sgkang
  - name: Rowan Cockett
date: 2018/08/19
---

In this example, we plot anomalous total magnetic field
from a magnetic dipole and pole targets. These targets are
excited by Earth magnetic fields.
We can vary the direction of the Earth magnetic field, and
magnetic moment of the target.

In [ ]:
import micropip
await micropip.install('ipywidgets')
await micropip.install('https://cdn.curvenote.com/pypi/geoana-0.5.0-py3-none-any.whl')

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.constants import mu_0, epsilon_0
from ipywidgets import interactive

from geoana import utils, spatial
from geoana.em import static

## Setup

define the location, orientation, and source, physical properties of the
wholespace and source parameters.

In [2]:
mu = mu_0  # permeability of free space (this is the default)
location = np.r_[0., 0., -10.]  # location of the dipole or pole

# dipole parameters
moment = 1

## Magnetostatic Dipole and Loop

Here, we build the geoana magnetic dipole and poie in a wholespace
using the parameters defined above.

In [3]:
def id_to_cartesian(inclination, declination):
    ux = np.cos(inclination/180.*np.pi)*np.sin(declination/180.*np.pi)
    uy = np.cos(inclination/180.*np.pi)*np.cos(declination/180.*np.pi)
    uz = -np.sin(inclination/180.*np.pi)
    return np.r_[ux, uy, uz]

def plot_amplitude(ax, x, y, v):
    plt.colorbar(
        ax.pcolormesh(
            x, y, v.reshape(len(x), len(y), order='F')
        ), ax=ax
    )
    ax.axis('square')
    ax.set_xlabel('y (east,  m)')
    ax.set_ylabel('x (north,  m)')

def plot_fields(inclination, declination):
    orientation = id_to_cartesian(inclination, declination)
    
    dipole = static.MagneticDipoleWholeSpace(
        location=location,
        orientation=orientation,
        moment=moment
    )
    
    pole = static.MagneticPoleWholeSpace(
        location=location,
        orientation=orientation,
        moment=moment
    )
    x = np.linspace(-36, 36, 100)
    y = np.linspace(-36, 36, 100)
    xyz = utils.ndgrid([x, y, np.r_[1.]])
    
    # evaluate the magnetic field
    b_vec_dipole = dipole.magnetic_flux_density(xyz)
    b_vec_pole = pole.magnetic_flux_density(xyz)
    b_total_dipole = dipole.dot_orientation(b_vec_dipole)
    b_total_pole = pole.dot_orientation(b_vec_pole)

    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    # plot dipole vector potential
    plot_amplitude(ax[0], x, y, b_total_dipole)
    # plot loop vector potential
    plot_amplitude(ax[1], x, y, b_total_pole)

    plt.show()

In [4]:
#| label: app:magnetic_dipole_whole_space
# inclination and declination (e.g. Vancouver)
interactive(plot_fields, inclination=(0.,67.,1.), declination=(0., 90.))

interactive(children=(FloatSlider(value=33.0, description='inclination', max=67.0, step=1.0), FloatSlider(valu…